# Supervised Training

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

%matplotlib inline

## Global Settings

In [2]:
LEFT_CENTER = (3, 3)
RIGHT_CENTER = (3, -2)